### All required installations

In [1]:
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 287 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 35.1 MB/s eta 0:00:01
     |████████████████████████████████| 574 kB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 121 kB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 8.4 MB/s  eta 0:00:01


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.13.3 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.1.0 which is incompatible.
     |████████████████████████████████| 2.4 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 147.1 MB 40.3 MB/s eta 0:00:01    |████████▏                       | 37.5 MB 440 kB/s eta 0:04:09     |██████████████████████▌         | 103.6 MB 37.3 MB/s eta 0:00:02     |████████████████████████████████| 146.7 MB 40.3 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 9.8 MB/s eta 0:00:01
     |██████████████████████████████

In [5]:
! git config --global credential.helper store

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /Users/vlad/.huggingface/token


In [6]:
! apt install git-lfs

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



### Pick a language to fine-tune XLS-R on

In [2]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "fr", split='train[10:20]')
common_voice_test = load_dataset("common_voice", "fr", split="test[10:20]")

Downloading:   0%|          | 0.00/19.1G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [1]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

NameError: name 'common_voice_train' is not defined

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))